<a href="https://colab.research.google.com/github/geethapriyan1692/NeuGenAI-Task/blob/main/Task_1_PDFCHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Install Required Libraries: Install necessary libraries like PyPDF2, FAISS, LangChain, and Transformers for the pipeline setup.

In [3]:
# Install required libraries
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain
!pip install transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.0 MB/s eta 0:00:00


In [4]:
import time
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from transformers import pipeline

2.Read PDF File: Load and extract text from the provided PDF file using PyPDF2.

In [5]:
# Path to the PDF file
pdf_path = "/content/EW.pdf"



*  After reading the pdf extracting the texts



In [6]:
# Read PDF and extract text
pdf = PdfReader(pdf_path)
raw_text = ''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content

3.Initialize Embeddings: Load the sentence-transformers/all-MiniLM-L6-v2 model to generate embeddings for the text chunks.

In [7]:
# Split the text into chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=300,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

4.Initialize Embeddings: Load the sentence-transformers/all-MiniLM-L6-v2 model to generate embeddings for the text chunks.

In [8]:
# Initialize Hugging Face Embeddings
model_id = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_id)

<ipython-input-8-eb5a3409aa1b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_id)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

5.Store Embeddings: Save the embeddings in a FAISS vector store for efficient similarity search.

In [9]:
# Generate embeddings for text chunks and store in FAISS vector store
document_store = FAISS.from_texts(texts, embedding=embeddings)

6.Load QA Model: Use the Hugging Face deepset/roberta-base-squad2 model for question-answering tasks.

In [10]:
# Load a question-answering pipeline from Hugging Face
qa_model_id = "deepset/roberta-large-squad2"
qa_pipeline = pipeline("question-answering", model=qa_model_id, tokenizer=qa_model_id)


config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


7.Query Function: Define a function to retrieve similar chunks, combine their text, and answer the query using the QA pipeline

In [11]:
# Function to handle queries

def query_pdf(query):
    # Search for similar documents in the FAISS vector store
    docs = document_store.similarity_search(query)
    input_text = " ".join(doc.page_content for doc in docs[:5])
    # Use the QA pipeline to get an answer
    answer = qa_pipeline(question=query, context=input_text)
    # Add context from top document to elaborate
    elaboration = docs[0].page_content[:300]  # Add the first 300 characters from the top doc
    return f"{answer['answer']} (Additional context: {elaboration})"


8.Interactive Querying: Accept user input for queries, retrieve answers, and display them.

In [12]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: whats the pdf is about
Answer: 
sheet (Additional context: drawings in order to help  in clarify ing 
the drawings and  to facilitat e the 
dimensioning  of drawings.  
The figure  shows  a regular multi -view 
drawing and a sectioned multi -view 
drawing of th e same part in the front 
view, the hidden features can be seen 
after sectioning.  
 Hatching  )


In [13]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: whats revision table
Answer: Revision Table , A revision table is normally located in the (Additional context:  the name of the drawing file  
 the name of the source part or assembly file  
 the units of the dimensions ( if the drawing is mistakenly printed on a different 
paper size, the scale becomes meaningless ) 
Revision Table , A revision table is normally located in the upper right of the drawing )


In [14]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: what is zoning
Answer:  (Additional context: zoning is used it is located inside the drawing frame.  Zoning  is usually used for large 
size drawing sheets  where it allows easy references to various parts of the drawing 
by referencing a coordinate such as B4. 
Notes , Information other than pictorial views and dimensions necessary for 
compl)
